## Södertälje
* [Issue #33](https://github.com/salgo60/Dogpark_Sweden/issues/33)
* [33_dogparks_södertälje](33_dogparks_södertälje.ipynb)
* [json](https://static-content.sodertalje.se/opendata/hundrastgardar.json) 


In [1]:
import time
import datetime  
start_time = time.time()
start_str = datetime.datetime.now().strftime("%Y-%m-%d %H:%M")
print(f"Started: {start_str}")


Started: 2025-10-13 17:22


In [2]:
import requests
import folium
import geopandas as gpd
from shapely.geometry import shape
from IPython.display import display

# 1️⃣ Fetch Södertälje open data (GeoJSON)
url = "https://static-content.sodertalje.se/opendata/hundrastgardar.json"
response = requests.get(url)
response.raise_for_status()
data = response.json()

# 2️⃣ Convert to GeoDataFrame
gdf = gpd.GeoDataFrame.from_features(data["features"], crs="EPSG:4326")

print("Kolumner:", gdf.columns.tolist())
print("Antal objekt:", len(gdf))
display(gdf.head())

# 3️⃣ Create a Folium map centered on Södertälje
m = folium.Map(location=[59.2, 17.63], zoom_start=12, tiles="OpenStreetMap")

# 4️⃣ Create layers
polygon_layer = folium.FeatureGroup(name="🐾 Hundrastgårdar (ytor)", show=True)
marker_layer = folium.FeatureGroup(name="📍 Hundrastgårdar (markörer)", show=True)

# 5️⃣ Iterate over each feature
for _, row in gdf.iterrows():
    name = row.get("Namn", "Okänt namn")
    web = row.get("Webbsida", "")
    addr = row.get("Adress", "—")
    area = row.get("Yta_m2", "—")
    geom = row.geometry
    centroid = geom.centroid
    lat, lon = centroid.y, centroid.x

    osm_url = f"https://www.openstreetmap.org/?mlat={lat:.6f}&mlon={lon:.6f}&zoom=17"

    popup_html = f"""
    <b>{name}</b><br>
    📍 <a href="{osm_url}" target="_blank">Visa på OpenStreetMap</a><br>
    🌐 <a href="{web}" target="_blank">Webbsida</a><br>
    🏠 Adress: {addr}<br>
    📏 Yta: {area} m²
    """

    # Add polygon (if geometry is area)
    if geom.geom_type in ["Polygon", "MultiPolygon"]:
        folium.GeoJson(
            data=geom.__geo_interface__,
            name=name,
            tooltip=name,
            popup=folium.Popup(popup_html, max_width=300),
            style_function=lambda x: {
                "fillColor": "green",
                "color": "darkgreen",
                "weight": 2,
                "fillOpacity": 0.4,
            },
        ).add_to(polygon_layer)

    # Add centroid marker
    folium.Marker(
        location=[lat, lon],
        popup=folium.Popup(popup_html, max_width=300),
        tooltip=name,
        icon=folium.Icon(color="darkgreen", icon="paw", prefix="fa")
    ).add_to(marker_layer)

# 6️⃣ Add layers to map
polygon_layer.add_to(m)
marker_layer.add_to(m)
folium.LayerControl(collapsed=False).add_to(m)

# 7️⃣ Display map
display(m)


Kolumner: ['geometry', 'Namn', 'Webbsida']
Antal objekt: 21


,geometry,Namn,Webbsida
0,POINT (17.67063 59.18967),"Kummelvägen, Östertälje",https://www.sodertalje.se/stad-och-trafik/natu...
1,POINT (17.62193 59.21003),"Hanstabruksskogen, Tallhöjden",https://www.sodertalje.se/stad-och-trafik/natu...
2,POINT (17.59566 59.20338),"Fornstigen, Ronna",https://www.sodertalje.se/stad-och-trafik/natu...
3,POINT (17.58919 59.19474),"Prästgårdsvägen, Geneta",https://www.sodertalje.se/stad-och-trafik/natu...
4,POINT (17.603 59.17849),"Gröndalsvägen, Hovsjö",https://www.sodertalje.se/stad-och-trafik/natu...


In [3]:
m.save("33_dogparks_södertälje.html")

In [4]:
from datetime import datetime 
# End timer and calculate duration
end_time = time.time()
elapsed_time = end_time - start_time# Bygg audit-lager för den här etappen

# Print current date and total time
print("Date:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
minutes, seconds = divmod(elapsed_time, 60)
print("Total time elapsed: {:02.0f} minutes {:05.2f} seconds".format(minutes, seconds))


Date: 2025-10-13 17:22:43
Total time elapsed: 00 minutes 01.03 seconds
